# Bell state quantum circuit
In this exercise we creates a two qubit circuit, with two qubits in superposition, and then measures the individual qubits, resulting in two coin toss results with the following possible outcomes with equal probability: $|00\rangle$, $|01\rangle$, $|10\rangle$, and $|11\rangle$. This is like tossing two coins.

We then take a look at using the CX (Controlled-NOT) gate to entangle the two qubits in a so called Bell state. This surprisingly results in the following possible outcomes with equal probability: $|00\rangle$ and $|11\rangle$. Two entangled qubits do not at all behave like two tossed coins.

We then run the circuit a large number of times to see what the statistical behavior of the qubits are.
Finally, we run the circuit on real IBM Q hardware to see how real physical qubits behave.

In this exercise we introduce the CX gate, which creates entanglement between two qubits, by flipping the controlled qubit (q_1) if the controlling qubit (q_0) is 1.
```
        ┌───┐     
q_0: |0>┤ H ├──■──
        └───┘┌─┴─┐
q_1: |0>─────┤ X ├
             └───┘
```


Import the required libraries, including the IBM Q library for working with IBM Q hardware.

In [ ]:
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.tools.monitor import job_monitor

# Import visualization
from qiskit.visualization import plot_histogram, plot_bloch_multivector

# Add the state vector calculation function
def get_psi(circuit): 
    global psi
    backend = Aer.get_backend('statevector_simulator') 
    psi = execute(circuit, backend).result().get_statevector(circuit)
    
#Load account information 
IBMQ.load_account()
provider = IBMQ.get_provider()

How many qubits do we want to use:

In [ ]:
n_qubits=int(input("Enter number of qubits:"))

Create quantum circuit that includes the quantum register and the classic register. Then add a Hadamard (super position) gate to all the qubits. Add measurement gates.

In [ ]:
qc1 = QuantumCircuit(n_qubits,n_qubits)

for qubit in range (0,n_qubits):
    qc1.h(qubit) #A Hadamard gate that creates a superposition
    qc1.measure(qubit,qubit)

print(qc1)

Now run the circuit with one shot. This represents a two-coin coin toss. Like before, we can also run the circuit more times to get statistics on the possible outcomes.
 

In [ ]:
backend = Aer.get_backend('qasm_simulator')

job = execute(qc1, backend, shots=1)
counts1  = job.result().get_counts(qc1)
print(counts1)
plot_histogram(counts1)

Let's look at a similar simulation result in Bloch sphere format.

In [ ]:
# Display the Bloch spheres
get_psi(qc1)
print(psi)
plot_bloch_multivector(psi)

Now we are going to do something different. We will entangle the qubits.

Create quantum circuit that includes the quantum register and the classic register. Then add a Hadamard (super position) gate to the first qubit. Then add a controlled-NOT gate (cx) between the first and second qubit, entangling them. Add measurement gates. 

In [ ]:
qc2 = QuantumCircuit(n_qubits,n_qubits)

qc2.h(0) # A Hadamard gate that puts the first qubit in superposition
for qubit in range (1,n_qubits):
    qc2.cx(0,qubit) #A controlled NOT gate that entangles the qubits.
    qc2.measure(qubit,qubit)

qc2.measure(0,0)
print(qc2)

Set the backend to a local simulator. Then create a quantum job for the circuit, the selected backend, that runs just one shot to simulate a coin toss with two simultaneously tossed coins, then run the job. Display the result; either 0 for up (base) or 1 for down (excited) for each qubit. Display the result as a histogram. Either |00> or |11> with 100% probability.

In [ ]:
backend = Aer.get_backend('qasm_simulator')

job = execute(qc2, backend, shots=1)
counts2  = job.result().get_counts(qc2)
print(counts2)
plot_histogram(counts2)

Let's look at a similar simulation result in Bloch sphere format.

In [ ]:
# Display the Bloch spheres
get_psi(qc2)
print(psi)
plot_bloch_multivector(psi)

Note how the two qubits completely agree. They are entangled. 

Now, lets run quite a few more shots, and display the statistsics for the two results: $|00\rangle$ and $|11\rangle$. This time, as we are no longer just talking about two qubits, but the amassed results of thousands of runs on these qubits, we can no longer use the Block sphere as a display tool.

In [ ]:
job = execute(qc2, backend, shots=1000)
result = job.result() 
counts  = result.get_counts()
print(counts)
plot_histogram(counts)

And look at that, we are back at our coin toss results, fifty-fifty. Every time one of the coins comes up heads (|0>) the other one follows suit. Tossing one coin we immediately know what the other one will come up as; the coins (qubits) are entangled.

**Important:** With the simulator we get perfect results, only |00> or |11>. On a real NISQ (Noisy Intermediate Scale Quantum computer) we do not expect perfect results like this. Let's run the Bell state once more, but on an actual IBM Q quantum computer.

Grab the least busy IBM Q backend.

In [ ]:
from qiskit.providers.ibmq import least_busy
backend = least_busy(provider.backends(operational=True, simulator=False))
#backend = provider.get_backend('ibmqx2')
print("Selected backend:",backend.status().backend_name)
print("Number of qubits(n_qubits):", backend.configuration().n_qubits)
print("Pending jobs:", backend.status().pending_jobs)

Lets run a large number of shots, and display the statistsics for the two results: $|00\rangle$ and $|11\rangle$ on the real hardware. Monitor the job and display our place in the queue.

In [ ]:
if n_qubits > backend.configuration().n_qubits:
    print("Your circuit contains too many qubits (",n_qubits,"). Start over!")
else:
    job = execute(qc2, backend, shots=1000)
    job_monitor(job)

Get the results, and display in a histogram. Notice how we no longer just get the perfect |00> and |11> entangled results, but also a few results that include non-entangled |01> and |10>. At this stage, quantum computers are not perfect calculating machines, but pretty noisy. 

In [ ]:
result = job.result()
counts  = result.get_counts(qc2)
print(counts)
plot_histogram(counts)

That was the simple readout. Let's take a look at the whole returned results:

In [ ]:
print(result)